# Study of Dice Rolls Probabilities with and without Concurrency.

In this notebook we will calculate the probability of outcome when rolling two dices at the same moment. We can
calculate exactly what are the probabilities of obtaining a particular result.
For example, if we have two dices with 6 sides, with faces numbered from 1 to 6, and we roll both dices at the same
moment, we can obtain the following possible outcomes:
2, 3, 4, 5, 6, ... , 12  
So if we enumerate exhaustively all possible combinations, we can calculate the probability distribution for each
outcome, the following function `dice_probabilities` calculates the probability distribution.

In [1]:
%precision 3
SIDES = 6

def dice_probabilities() -> list:
    """
    Returns an array of probabilities to obtain a certain result when rolling two dices.
    """
    dist = [0 for _ in range(2 * SIDES + 1)]
    for i in range(1, SIDES + 1):
        for j in range(1, SIDES + 1):
            dist[i + j] += 1
    for k in range(len(dist)):
        dist[k] /= SIDES * SIDES
    return dist

As we can see, there is no probability of obtaining 0 or 1 as result, because the minimum result is 2.

In [2]:
expected_probabilities = dice_probabilities()
print(expected_probabilities)

[0.0, 0.0, 0.027777777777777776, 0.05555555555555555, 0.08333333333333333, 0.1111111111111111, 0.1388888888888889, 0.16666666666666666, 0.1388888888888889, 0.1111111111111111, 0.08333333333333333, 0.05555555555555555, 0.027777777777777776]


Also, we can check that the probability distribution adds up to 1, so our calculation is in principle correct.

In [3]:
sum(expected_probabilities)

1.000

## Rolling Dices
Now we can check if this probability distribution is correct by simulating the process of rolling the dices with a
computer. First we will do it using a serial approach, without multiprocessing, after that, we will use a concurrent
approach with Python multiprocessing module, then we will compare both to see how concurrency improves this kind of
cpu-bound calculations.

We need to import some modules to help us with random number generation, rounding, and timing the algorithm runtime.

In [4]:
import random
import math
import time

The function `roll_dices` will roll a specified number of times and accumulate the results in the `dice_counts` list,
then we will divide the counts by the total number of rolls to obtain the actual probability distribution.

In [5]:
def roll_dices(number_rolls):
    dice_counts = [0 for _ in range(2 * SIDES + 1)]
    for i in range(number_rolls):
        dice1 = random.randint(1, SIDES)
        dice2 = random.randint(1, SIDES)
        dice_counts[dice1 + dice2] += 1
    return [dice_count / number_rolls for dice_count in dice_counts]

Now we run our experiment with 10 million rolls, also we time the execution to compare with concurrency later.

In [6]:
roll_count = 10000000
print(f'Roll count: {roll_count}')
start_time = time.time()
actual_probabilities = roll_dices(roll_count)
end_time = time.time()
serial_running_time = (end_time - start_time) * 1000
print(f'--- {serial_running_time:.3f} ms ---') 
print(actual_probabilities)

Roll count: 10000000
--- 13560.353 ms ---
[0.0, 0.0, 0.0276678, 0.055521, 0.0834503, 0.110929, 0.1390298, 0.166541, 0.1390419, 0.1110754, 0.0833736, 0.0555205, 0.0278497]


As you can see the execution of this algorithm took approx. 14 seconds to complete, without concurrency (using only
1-core of the machine).
Now lets check if the probabilities look good.

In [7]:
sum(actual_probabilities)

1.000

The probabilities sum is ok, so now let's calculate the cumulative difference between the expected probabilities and
the actual probabilities.

In [8]:
cumulative_diff = sum([abs(actual_probabilities[i] - expected_probabilities[i]) for i in range(len(actual_probabilities))])
print(f'Cumulative Diff: {cumulative_diff:.3f}')

Cumulative Diff: 0.001


The difference is very small, so we can see that our initial calculation was correct, now we can move onto the
concurrent implementation, to see if we can make this algorithm faster.

## Concurrent Dice Rolling with Python Multiprocessing

Why we use multiprocessing and not multithreading? The Python interpreter has a Global Interpreter Lock (GIL)
that prohibits multiple threads to execute python instructions. So if you use multithreading for cpu-bound processing
like in this case, you will have thread contention, and the execution will be serial, not parallel, so you will lose the
benefits of concurrency. For IO-bound processing, for example, reading multiple files or socket communications,
threading is a good alternative, because each thread will wait for IO events at the same moment.

To allow multiprocessing to execute parallel tasks inside a Jupyter Notebook, it's better to have the processing task,
in this case `roll_dices_task`, in a separate external module (`dice_multiprocessing`), so the multiprocessing module
will find the function to execute. If you define the function inside the Jupyter Notebook, the multiprocessing module
won't be able to find the function to execute, and it will raise an exception.

We will need:
- `roll_dices_task`: this function will simulate the dice rolling process.
- `Process`: class to execute tasks.
- `Array`: this class will allow us to create arrays that can be shared with the parallel processes, these arrays will
store the probability distributions, one per process.
- `cpu_count`: this function returns the number of hardware cpus or cores that are available for multiprocessing. It's
advised to execute just one process per core, to prevent serialization of tasks.

In [9]:
from dice_multiprocessing import roll_dices_task
from multiprocessing import Process, Array, cpu_count

Let's see the definition of the `roll_dices_task`, we will see that is almost the same as the previous `roll_dices`
function. The difference is that we use an `Array` argument instead of a list, also we return the dice counts instead
of the probabilities, because we will add these counts in the main process logic and then calculate the probability
distribution.

```
def roll_dices_task(rolls_per_process: int, dice_counts: Array):
    for i in range(rolls_per_process):
        dice1 = random.randint(1, SIDES)
        dice2 = random.randint(1, SIDES)
        dice_counts[dice1 + dice2] += 1
```

We calculate the number of rolls that will be executed per process.

In [10]:
number_processes = cpu_count()
roll_count = 10000000
print(f'Roll count: {roll_count}')
rolls_per_process = math.ceil(roll_count / number_processes)
print(f'Roll per process: {rolls_per_process}')

Roll count: 10000000
Roll per process: 625000


Then we create an array of `Array` objects that will be used to store the probability distribution of each of the
running tasks. The arguments are as follows:
- 1st argument: the datatype, in this case an unsigned int (we are counting the number of rolls)
Please check the [arrays](https://docs.python.org/3/library/array.html#module-array) module for reference on data types.
- 2nd argument: is the initial values used to fill the array, a scalar or a sequence.
- 3rd argument: `lock=False` means that the array will be used for communication between the parent process
(this notebook process) and the tasks, and we don't expect to have concurrent access to the same array from multiple
concurrent processes. It's better to enable lock only when it's necessary, because acquiring the lock consumes
considerable time, and this will slow down the concurrent execution making the multiprocessing strategy slower than
using a serial approach.

In [11]:
results = [Array('I', [0 for _ in range(2 * SIDES + 1)], lock=False) for _ in range(number_processes)]

We need to create each process object specifying the task to execute and the arguments. In this case:
- `roll_dices_task`: this function defined in an external module will simulate the rolling dices using python `random`
module.
- `rolls_per_process`: specifies how many times the dices will roll.
- `results[i]`: this is the corresponding `Array` object that will be used to store the probability distributions.

In [12]:
processes = [Process(target=roll_dices_task, args=(rolls_per_process, results[i])) for i in range(number_processes)]

After creating the process objects we need to call `start` to begin the execution of each task. Also, we need to pause
the notebook process and resume only after each task is finish, for these, we use the `join` method, this will wait
until the process is finished and then continue execution.
Finally, we calculate the running time to compare with the serial approach.

In [13]:
start_time = time.time()
for process in processes:
    process.start()
for process in processes:
    process.join()
end_time = time.time()
multiprocessing_running_time = (end_time - start_time) * 1000
print(f'--- {multiprocessing_running_time:.3f} ms ---')

--- 2318.639 ms ---


We can see that multiprocessing is way faster than the serial approach, when it's used without synchronization
mechanisms like mutexes. In this specific example, is more than 6 times faster than the serial approach, on my 16 core
machine.
If we extrapolate to a calculation of several hours, multiprocessing or multithreading can save a lot of time
and processing resources (cost savings if you're renting a cloud service)

In [14]:
serial_running_time / multiprocessing_running_time

5.848

Finally, let's check if the results are correct by comparing to the probability distribution with the previously
calculated distribution.

In [15]:
dice_counts = [0 for _ in range(2 * SIDES + 1)]
for i in range(len(dice_counts)):
    dice_counts[i] = 0
    for result in results:
        dice_counts[i] += result[i]
print(dice_counts)

[0, 0, 278332, 554878, 834009, 1112562, 1387877, 1665783, 1390180, 1110856, 832094, 555367, 278062]


In [16]:
actual_probabilities = [0 for _ in range(2 * SIDES + 1)]
for i in range(len(dice_counts)):
    actual_probabilities[i] = dice_counts[i] / roll_count
print(actual_probabilities)

[0.0, 0.0, 0.0278332, 0.0554878, 0.0834009, 0.1112562, 0.1387877, 0.1665783, 0.139018, 0.1110856, 0.0832094, 0.0555367, 0.0278062]


We can see that the cumulative difference is very low, so the multiprocessing approach didn't affect the general result,
only made the calculation faster.

In [17]:
diff = sum([abs(actual_probabilities[i] - expected_probabilities[i]) for i in range(len(actual_probabilities))])
print(f'Cumulative Diff: {diff}')

Cumulative Diff: 0.0008512222222221769


## Conclusions
- This experiment serves as an example of how multiprocessing can decrease the running time of an algorithm by
exploiting concurrency, but we must be careful that it doesn't replace a good algorithm design. It's known that a better
algorithm design will win over increasing the number of cores or execution speed of a machine.
- We need to be careful with multiprocessing and synchronization mechanisms like mutexes (Locks on Python docs), because
a mutex or lock can decrease the speed considerably, slowing the execution of the multiprocessing strategy to a point
that is possible to run slower than a serial strategy.
- When the dataset to process is small, there is no point in using multiprocessing, in this case we had 10 million
rolls, but if we had less, it's better to use a serial strategy.